# Visualization of flash response
This notebook allow to explore flash response and its features.
Data must be created using [PRO_get_features_of_flash_response](PRO_get_features_of_flash_response.ipynb)

In [1]:
import h5py
from ipywidgets import IntSlider, interact, Dropdown
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook 

In [2]:
# source_file = '../data/processed_protocols/MR-0092t2_analysis_of_protocols_150um_merge.hdf5'
source_file = '../data/processed_protocols/MR-0092t2_modified_analysis_of_protocols_150um_merge.hdf5'
output_file = '../fig/flash/'

In [3]:
intensities = [50, 100, 150, 200, 255]
panalysis = h5py.File(source_file, 'r')

prefix = 'nd2'
type_group = '/flash/{}_255/type/'.format(prefix)

no_valid_units = []
valid_units = []
all_valid_units = []
for key in panalysis[type_group]:
    all_valid_units.append(key)
    if panalysis[type_group+key][...]:
        valid_units.append(key)
    else:
        no_valid_units.append(key)
print('Numbers of valid units: {:d}\nNumber of not valid unit: {:d}'.format(len(valid_units),len(no_valid_units)))

Numbers of valid units: 624
Number of not valid unit: 130


# Visualization flash response for one ND=(2,3,4,5) and all intensities together

In [4]:
def plot_fit(key):
    group = '/flash/{}'.format(prefix) + '_{:03}/est_psth/'
    dset = '/flash/{}'.format(prefix) + '_{:03}/est_psth/{}'
    
    plt.figure(figsize=(10,6))
    for kintensity in intensities:
        time = panalysis[group.format(kintensity)].attrs['time']
        response = panalysis[dset.format(kintensity,key)][...]
        s_on, f_on, s_off, f_off = panalysis[group.format(kintensity)].attrs['bounds']
        plt.plot(time, response, label=str(kintensity), alpha=0.5)
        max_amp = response.max()
        plt.vlines(s_off,0,max_amp)
    plt.xlabel('time [s]')
    plt.ylabel('instantaneous firing rate [spk/bins]')
    plt.title('Flash response for {} in different intensities. Filter {}'.format(key, prefix.upper()))
    plt.legend()
    plt.show()
    plt.close()

In [5]:
prefix = 'nd5'
# interact(plot_fit, kidx=IntSlider(min=0, max=len(temporal_sta)-1, step=1, value=172));
interact(plot_fit, key=Dropdown(options=all_valid_units));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUna2V5Jywgb3B0aW9ucz0odSd0ZW1wXzAnLCB1J3RlbXBfMScsIHUndGVtcF8xMCcsIHUndGVtcF8xMDAnLCB1J3TigKY=


# Visualization flash response for ND(2,3,4,5) and all intensities together

In [6]:
def plot_nd(key):
    fig, ax = plt.subplots(4,1,figsize=(10,15), sharex=True, sharey=True)
    for kfilter, kax in zip(['nd2', 'nd3', 'nd4', 'nd5'], ax.flatten()):
        group = '/flash/{}'.format(kfilter) + '_{:03}/est_psth/'
        dset = '/flash/{}'.format(kfilter) + '_{:03}/est_psth/{}'
        for kintensity in intensities:
            time = panalysis[group.format(kintensity)].attrs['time']
            response = panalysis[dset.format(kintensity,key)][...]
            s_on, f_on, s_off, f_off = panalysis[group.format(kintensity)].attrs['bounds']
            kax.plot(time, response, label=str(kintensity), alpha=0.5)
            max_amp = response.max()
            kax.vlines(s_off,0,max_amp)
        kax.set(xlabel='time [s]', ylabel='instantaneous firing rate [spk/bins]',
               title='Filter {}'.format(kfilter.upper()))
        kax.legend()
        kax.grid(b=True, which='both', ls=':', alpha=0.3)
#         kax.grid(which='major', ls=':', alpha=0.3)
    fig.suptitle('Flash response for {} in different intensities.'.format(key))
#     plt.tight_layout()
    plt.show()
    plt.close()

In [7]:
# interact(plot_fit, kidx=IntSlider(min=0, max=len(temporal_sta)-1, step=1, value=172));
interact(plot_nd, key=Dropdown(options=all_valid_units));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUna2V5Jywgb3B0aW9ucz0odSd0ZW1wXzAnLCB1J3RlbXBfMScsIHUndGVtcF8xMCcsIHUndGVtcF8xMDAnLCB1J3TigKY=


# Visualization all intensities for an ND(2,3,4,5)

In [8]:
prefix = 'nd2'

In [6]:
def plot_intensities(key):
    pre_name = '/flash/{}'.format(prefix)
    group = pre_name + '_{:03}/est_psth/'
    dset = pre_name + '_{:03}/est_psth/{}'
    flash_type = pre_name + '_{:03}/type/{}'
    flash_char = pre_name + '_{:03}/char/{}'
    fig, ax = plt.subplots(3, 2, figsize=(15,12), sharex=True, sharey=True)
    fig.suptitle('Flash response for {} in different intensities. Filter {}'.format(key, prefix.upper()))
    for kintensity, kax in zip(intensities, ax.flatten()):
        time = panalysis[group.format(kintensity)].attrs['time']
        response = panalysis[dset.format(kintensity,key)][...]
        s_on, f_on, s_off, f_off = panalysis[group.format(kintensity)].attrs['bounds']
        chars = panalysis[flash_char.format(kintensity, key)][...]
        kax.plot(time, response, label=str(kintensity), alpha=0.5)
        max_amp = response.max()
        kax.vlines(s_off,0,max_amp)
        kax.vlines(chars[0]+s_on, 0, max_amp, alpha=0.5, color='r', linestyle='--')
        kax.vlines(chars[1]+s_off, 0, max_amp, alpha=0.5, color='b', linestyle='--')
        flash_t = panalysis[flash_type.format(kintensity,key)][...]
        kax.set(xlabel='time [s]', ylabel='inst fr [spk/bins]', 
               title='Type {}'.format(kintensity),
               )
        kax.legend()
    plt.show()
    plt.close()

In [7]:
# interact(plot_fit, kidx=IntSlider(min=0, max=len(temporal_sta)-1, step=1, value=172));
interact(plot_intensities, key=Dropdown(options=valid_units));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUna2V5Jywgb3B0aW9ucz0odSd0ZW1wXzAnLCB1J3RlbXBfMScsIHUndGVtcF8xMCcsIHUndGVtcF8xMDAnLCB1J3TigKY=


# Close HDF5 file

In [9]:
panalysis.close()